# CAD Challenge 1: Binary classification

## Set things up

### Set dataset location

In [3]:
dataset_path = 'd:\\Files\\Projects\\MAIA\\UdG\\CAD\\datasets\\challenge_1_binary'

### Imports

In [ ]:
# matplotlib
import matplotlib.pyplot as plt

### Set up matplotlib

In [2]:
plt.rcParams['figure.figsize'] = [10, 5]
%matplotlib inline
#%matplotlib widget

hello world
